In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
# 2. Load dataset
df = pd.read_csv('../data/processed/avg_combined_dataset.csv')
df = df.sort_values(by=['place_name', 'year']).reset_index(drop=True)
df.head()

In [ ]:
plt.figure(figsize=(10, 4))
for place in df['place_name'].unique():
    subset = df[df['place_name'] == place]
    plt.plot(subset['year'], subset['total_visitor'], marker='o', label=place)

plt.title("Total Visitor per Place Over Years")
plt.xlabel("Year")
plt.ylabel("Total Visitor")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 5. Modeling ARIMA + LSTM per tempat wisata
results = []

for place in df['place_name'].unique():
    print(f"\n=== Modeling: {place.upper()} ===")
    data = df[df['place_name'] == place].copy()
    series = data['total_visitor'].values
    years = data['year'].values

    ## ------------------------ ARIMA ------------------------
    model_arima = ARIMA(series, order=(1, 1, 1))
    fit_arima = model_arima.fit()
    forecast_arima = fit_arima.forecast(steps=1)
    print(f"ARIMA Forecast 2025: {int(forecast_arima[0])}")

    ## ------------------------ LSTM ------------------------
    scaler = MinMaxScaler()
    scaled_series = scaler.fit_transform(series.reshape(-1, 1))

    X_lstm, y_lstm = create_lstm_dataset(scaled_series, n_steps=2)
    X_lstm = X_lstm.reshape((X_lstm.shape[0], X_lstm.shape[1], 1))

    model_lstm = Sequential()
    model_lstm.add(LSTM(50, activation='relu', input_shape=(2, 1)))
    model_lstm.add(Dense(1))
    model_lstm.compile(optimizer='adam', loss='mse')
    model_lstm.fit(X_lstm, y_lstm, epochs=200, verbose=0)

    # Predict next step (2025)
    last_steps = scaled_series[-2:].reshape((1, 2, 1))
    pred_scaled = model_lstm.predict(last_steps, verbose=0)
    forecast_lstm = scaler.inverse_transform(pred_scaled)[0][0]
    print(f"LSTM Forecast 2025: {int(forecast_lstm)}")

    # Simpan hasil
    results.append({
        'place_name': place,
        'arima_2025': int(forecast_arima[0]),
        'lstm_2025': int(forecast_lstm)
    })


In [ ]:
def create_sequence(data, step=3):
    X, y = [], []
    for i in range(len(data) - step):
        X.append(data[i:i+step])
        y.append(data[i+step])
    return np.array(X), np.array(y)

sequence_step = 3
X, y = create_sequence(scaled_data, sequence_step)

In [ ]:
# 6. Tampilkan hasil gabungan
forecast_df = pd.DataFrame(results)
forecast_df